# Chapter 8: Time Series Analysis

In finance and economics, a huge amount of our data is in the format of time-series,
such as stock prices and Gross Domestic Products (GDP). From Chapter 4, Sources
of Data, it is shown that from Yahoo!Finance, we could download daily, weekly, and
monthly historical price time-series. From Federal Reserve Bank's Economics Data
Library (FRED), we could retrieve many historical time-series such as GDP. For
time-series, there exist many issues, such as how to estimate returns from historical
price data, how to merge datasets with the same or different frequencies, seasonality,
and detect auto-correlation. Understanding those properties is vitally important for
our knowledge development.

In this chapter, the following topics will be covered:
• Introduction to time-series analysis

• Design a good date variable, and merging different datasets by date

• Normal distribution and normality test

• Term structure of interest rates, 52-week high, and low trading strategy

• Return estimation and converting daily returns to monthly or annual returns

• T-test, F-test, and Durbin-Watson test for autocorrelation

• Fama-MacBeth regression

• Roll (1984) spread, Amihud's (2002) illiquidity, and Pastor and Stambaugh's
(2003) liquidity measure

• January effect and weekday effect

• Retrieving high-frequency data from Google Finance and Prof. Hasbrouck's
TORQ database (Trade, Order, Report, and Quotation)

• Introduction to CRSP (Center for Research in Security Prices) database

## Introduction to Time Series

Most finance data is in the format of time-series, see the following several examples.
The first one shows how to download historical, daily stock price data from
Yahoo!Finance for a given ticker's beginning and ending dates:

In [1]:
import yfinance as yf
x = yf.download('IBM', start="2016-01-01", end="2016-01-21", rounding=True)
print(x)

[*********************100%***********************]  1 of 1 completed
              Open    High     Low   Close  Adj Close    Volume
Date                                                           
2016-01-04  129.64  129.99  128.34  129.97      89.53   5469952
2016-01-05  130.75  130.87  128.92  129.88      89.47   4105341
2016-01-06  128.47  129.62  127.74  129.23      89.02   4509201
2016-01-07  127.82  129.08  126.61  127.02      87.50   7348987
2016-01-08  127.32  127.93  125.54  125.84      86.69   4981784
2016-01-11  126.01  127.93  125.97  127.37      87.74   5203222
2016-01-12  127.77  127.90  125.36  127.06      87.53   5312320
2016-01-13  127.63  128.37  125.33  125.40      86.39   4916305
2016-01-14  126.13  127.91  125.45  127.07      87.53   5972242
2016-01-15  124.29  125.12  123.20  124.31      85.64   9422891
2016-01-19  124.39  126.29  122.43  122.48      84.37  10438662
2016-01-20  113.25  118.54  112.81  116.50      80.25  16901059


Let's see if webreader still works. EDIT: IT doesn't.